<!-- Logo al 25% -->
<td width="45%" align="left" valign="middle">
  <img src="https://www.upc.edu/comunicacio/ca/identitat/descarrega-arxius-grafics/fitxers-marca-principal/upc-positiu-p3005.png" width="300">
</td>

<!-- Texto al 75%, alineado a la derecha -->
<td width="5%" align="right" valign="middle">
  <p style="margin: 0;"><b>Intelligence Data Science and Artificial Intelligence (IDEAI)</b></p>
  <p style="margin: 0;"><b>Grau en Estadística (UB - UPC)</b></p>
  <p style="margin: 0;">Mètodes Estadístics per la Mineria de Dades (MeMDa)</p>
</td>

# 🧠 **Transfer Learning: Aplicar estilo a las imagenes**

 ### 🔧 1. Instalación

In [ ]:
!pip install diffusers==0.32.0 transformers accelerate safetensors torch torchvision --quiet

### 🧠 2. Cargar modelos por estilo (img2img)

Usamos `AutoPipelineForImage2Image` y un diccionario por estilo:

In [ ]:
import torch
from diffusers import AutoPipelineForImage2Image

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Usando dispositivo:", device)

STYLE_MODELS = {
    "pixar": {
        "model_id": "nitrosocke/mo-di-diffusion",
        "extra_prompt": "modern disney style, 3d animated movie still"
    },
    "simpsons": {
        "model_id": "Norod78/sd-simpsons-model",
        "extra_prompt": "simpsons style, yellow skin, flat colors, cartoon lineart, tv show screenshot"
    },
    "anime": {
        "model_id": "gsdf/Counterfeit-V2.5",
        "extra_prompt": "anime style, detailed eyes, cel shading"
    },
    "funko": {
        "model_id": "runwayml/stable-diffusion-v1-5",
        "extra_prompt": "Funko Pop figure, big head, small body, 3d vinyl toy"
    },
}

# Caché de pipelines para no descargar varias veces
PIPELINES_CACHE = {}

def get_pipeline_for_style(style: str):
    if style not in STYLE_MODELS:
        raise ValueError(f"Estilo '{style}' no soportado. Usa uno de: {list(STYLE_MODELS.keys())}")

    if style in PIPELINES_CACHE:
        return PIPELINES_CACHE[style]

    model_id = STYLE_MODELS[style]["model_id"]
    print(f"Cargando modelo para estilo '{style}': {model_id}")

    pipe = AutoPipelineForImage2Image.from_pretrained(
        model_id,
        torch_dtype=torch.float16 if device == "cuda" else torch.float32,
        safety_checker=None
    ).to(device)

    PIPELINES_CACHE[style] = pipe
    return pipe


Si alguno de estos modelos te pide aceptar la licencia de Stable Diffusion en Hugging Face, tendrás que haberlo aceptado en tu cuenta y (si hace falta) usar un token en Colab.

**Modelo base vs. modelo afinado**

* El Stable Diffusion “normal” sabe dibujar un poco de todo, pero no domina 100% un estilo concreto (Simpsons, Pixar...)

* Cuando quieres un estilo muy concreto, es mejor usar un **modelo ya entrenado específicamente** con imágenes de esa serie/estudio (por eso usamos `mo-di-diffusion` para estilo Disney/Pixar, `sd_asim_simpsons` para Simpsons, etc.).



### 🖼️ 3. Subir la foto de la persona

In [ ]:
from google.colab import files
from PIL import Image

uploaded = files.upload()  # elige una foto de una persona

image_name = list(uploaded.keys())[0]
init_image = Image.open(image_name).convert("RGB")

# Redimensionamos a algo razonable (muchos modelos están entrenados a 512x512)
init_image = init_image.resize((512, 512))
display(init_image)

### 🎨 4. Función genérica de estilizado

In [ ]:
import random

NEGATIVE_PROMPT_COMMON = (
    "blurry, low quality, deformed, bad anatomy, extra limbs, distorted face, artifacts, grainy, ugly"
)

def stylize_person(
    init_image: Image.Image,
    style: str,
    strength: float = 0.65,
    guidance_scale: float = 7.5,
    seed: int | None = None,
    num_inference_steps: int = 30,
    extra_text: str = "",
):
    """
    Aplica un estilo cartoon a la foto de una persona usando diferentes modelos por estilo.
    - strength ~0.4-0.6  -> respeta bastante la cara
    - strength ~0.7-0.9  -> cambia mucho (muy cartoon)
    """
    pipe = get_pipeline_for_style(style)
    style_info = STYLE_MODELS[style]

    if seed is None:
        seed = random.randint(0, 10_000)

    generator = torch.Generator(device=device).manual_seed(seed)

    base_prompt = (
        "portrait of the same person, centered, looking at the camera, "
        "clear facial features, high quality"
    )

    full_prompt = base_prompt + ", " + style_info["extra_prompt"]
    if extra_text:
        full_prompt += ", " + extra_text

    print("Prompt usado:\n", full_prompt)

    image = pipe(
        prompt=full_prompt,
        image=init_image,
        strength=strength,
        guidance_scale=guidance_scale,
        num_inference_steps=num_inference_steps,
        negative_prompt=NEGATIVE_PROMPT_COMMON,
        generator=generator,
    ).images[0]

    return image


### ▶️ 5. Probar estilos concretos

Aquí puedes ir cambiando el estilo y parámetros hasta que veas algo claramente cartoon.

* **Parámetro** `strength`
  * Si es muy bajo (~0.3-0.4): casi no cambia la imagen -> parece "no hace nada"
  * Si es medio (~0.5-0.7): se nota el estilo pero mantiene bastante la cara.
  * Si es alto (~0.8-0.9): se vuelve muy cartoon, pero puede perder la identidad

* **Parámetro** `guidance_scale`:
  * Valores altos (7-9) = el modelo obedece mucho al prompt (más estilo)
  * Valores bajos (4-6) = mezcla más con la foto original, menos dramático.

* **Parámetro** `num_inference_steps`:
  

### Ejemplo 1: “Pixar / Disney”

In [ ]:
from IPython.display import display

out_pixar = stylize_person(
    init_image,
    style="pixar",
    strength=0.7,        # más alto para que se note el 3D cartoon
    guidance_scale=8.5,  # más fuerte el prompt
    seed=1234,
)
display(out_pixar)
out_pixar.save("persona_pixar.png")

### Ejemplo 2: “Simpsons”

In [ ]:
out_simpsons = stylize_person(
    init_image,
    style="simpsons",
    strength=0.75,       # Simpsons necesita bastante deformación
    guidance_scale=8.5,
    seed=1234,
)
display(out_simpsons)
out_simpsons.save("persona_simpsons.png")

### Ejemplo 3: “Anime”

In [ ]:
out_anime = stylize_person(
    init_image,
    style="anime",
    strength=0.65,
    guidance_scale=8.0,
    seed=1234,
)
display(out_anime)
out_anime.save("persona_anime.png")

### Ejemplo 4: “Funko”

In [ ]:
out_funko = stylize_person(
    init_image,
    style="funko",
    strength=0.7,
    guidance_scale=8.0,
    seed=1234,
)
display(out_funko)
out_funko.save("persona_funko.png")
